In [ ]:
import ee
ee.Initialize(project='pyregence-ee')
#import pprint
import datetime
import scripts.analysis_functions as af
import scripts.utils as utils
%load_ext autoreload
%autoreload 2

### Run BS Mapper for a particular year

#### Testing cell, ignore

In [12]:
#testing python / GEE. To be deleted.

# fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406')
# irwin_list = ['56C3E0AE-98EF-4433-863D-23A5DFF3D0C2', '60146AC0-3AC8-462B-AC80-60D31F1B4D96', 
#               '4CE00061-B897-49DC-8839-D08E040FF65D', 'BDE833A4-B278-4418-8526-67F2BFC7D7CD'] 
#     #BOBCAT = 56C3E0AE-98EF-4433-863D-23A5DFF3D0C2, DOME = 60146AC0-3AC8-462B-AC80-60D31F1B4D96, 
#     #CASTLE = 4CE00061-B897-49DC-8839-D08E040FF65D (note multipolygon), SILVERADO = BDE833A4-B278-4418-8526-67F2BFC7D7CD (late fire 2020-10-26, should be filtered out)
# fires = fires_raw.filter(ee.Filter.inList('irwinID',irwin_list))

# meta = {
#     'this_a':'fjdksa',
#     'this_b':'a;lwoeri'
# }

# fires.set(meta)

# fires.set('here','askd')

# print(fires.propertyNames().getInfo())

# fire_irwin_list = fires.aggregate_array('irwinID')
# print(fire_irwin_list.getInfo())

# fire_name_list = fires.aggregate_array('Incid_Name')
# print(fire_name_list.getInfo())

# def set_id_test(feat: ee.Feature):
#     fire_name = ee.String(feat.get('Incid_Name'))
#     sep = ee.String('_')
#     fire_irwin = ee.String(feat.get('irwinID'))
#     fire_intermed = fire_name.cat(sep)
#     fire_id = fire_intermed.cat(fire_irwin)
#     return feat.set('fire_id', fire_id)

# fires_id = fires.map(set_id_test)
# id_list = fires_id.aggregate_array('fire_id')
# print(id_list.getInfo())


#fire_band_list = fire_name_list.cat(fire_irwin_list)
#print(fire_band_list.getInfo())

#print(fires_raw.first().getString('Discovery')) #ee.String
#print(fires_raw.first().getString('Discovery').getInfo())

#print(ee.Date(fires_raw.first().getString('Discovery')))



# print(fires_raw.size().getInfo())

# should_zero = fires_raw.filter(ee.Filter.notNull(['nonesuch']))
# print(should_zero.size().getInfo())

# fires_raw.set('this_property', 'something')
# print(fires_raw.)
# should_all = fires_raw.filter(ee.Filter.notNull(['this_property']))
# print(should_all.size().getInfo())

# nonesuch = fires_raw.get('nonesuch')
# print(nonesuch)

# if nonesuch is None:
#     print('yup, not here')

#print(datetime.now())
#print(datetime.datetime.now())

# now = ee.Date(datetime.datetime.now())
# then = ee.Date(datetime.datetime(2023, 8, 31))

# print(now.difference(then, 'days').getInfo())
# print(then.difference(now, 'days').getInfo())


# sim_date_list = [datetime.datetime(2020, 9, 30), 
#                  datetime.datetime(2020, 12, 31), 
#                  datetime.datetime(2021, 3, 31), 
#                  datetime.datetime(2021, 6, 30)]

# sim_date = sim_date_list[0] #possibly will make a loop, setting manual for now
# sim_date_ee = ee.Date(sim_date)

# #For consistency, foramt the sim date the same as Discovery date as from pull_mtbs_fires.py
# #    gdf.loc[:,'Discovery'] = gdf.loc[:,'Discovery'].apply(lambda x: str(x)[0:10])
# #    gdf.loc[:,'Discovery'] = gdf.loc[:,'Discovery'].astype('str')
# sim_date_formatted = str(sim_date)[0:10]
# print(sim_date_formatted)



['system:version', 'system:id', 'system:asset_size']


#### Simulation runs

In [35]:
# For running a year of fires with a SIMULATED run date (as if run on this date)
# REQUIRES USER INPUT/SELECTION, see starred section below. 
#  Uses specialized subfunctions that are variants of original functions
#  Developed using MTBS fires, would need modification for NIFC (most likely, unless field names matched)

# Last edited 2023-09-11 by dnekorchuk

#***
# USER: 
# 1. Set export flag (all exports)
# 2. Enter year value (for labeling)
# 3. Select simulation date
# 4. Align data import with year value
# 5. Change out folder if needed
# 6. Note that fire filter date is set to first sim date for all sims, change if needed.
#  Developed with MTBS, will likely need modifications for NIFC
#***

export_flag = False
csv_export_flag = False
yr = '2020' #align with fires import below
sim_date_list = [datetime.datetime(2020, 9, 30), 
                 datetime.datetime(2020, 12, 31), 
                 datetime.datetime(2021, 3, 31), 
                 datetime.datetime(2021, 6, 30)]
sim_index = 0 #pick 0 - 3 for sept, dec, march, june simulation
    #setting manually, considered a loop, but not worth the effort especially if need to rerun specific ones

##### IMPORT DATA #####
# One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch

# ### OPTION 1: MTBS option
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406")
# data_origin = 'mtbs'
# asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

### OPTION 2: Test / development using filtered MTBS, limited numbers of fires
fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406')
irwin_list = ['56C3E0AE-98EF-4433-863D-23A5DFF3D0C2', '60146AC0-3AC8-462B-AC80-60D31F1B4D96', 
              '4CE00061-B897-49DC-8839-D08E040FF65D', 'BDE833A4-B278-4418-8526-67F2BFC7D7CD'] 
    #BOBCAT = 56C3E0AE-98EF-4433-863D-23A5DFF3D0C2, DOME = 60146AC0-3AC8-462B-AC80-60D31F1B4D96, 
    #CASTLE = 4CE00061-B897-49DC-8839-D08E040FF65D (note multipolygon), SILVERADO = BDE833A4-B278-4418-8526-67F2BFC7D7CD (late fire 2020-10-26, should be filtered out)
fires = fires_raw.filter(ee.Filter.inList('irwinID',irwin_list))
data_origin = 'mtbs4'
#output folder
asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

#Probably does not work with NIFC, unless various property names match. 
    # ### OPTION x: NIFC option
    # fires_asset = f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte100ac_20230406"
    # fires = ee.FeatureCollection(fires_asset)
    # data_origin = 'nifc'
    # asset_folder = 'projects/pyregence-ee/assets/conus/nifc'


##### METADATA COLLECTION 1 #####

# Metadata collection and print
fires_count = fires.size().getInfo()
print(f'Total Fires in FeatureCollection: {fires_count}')

##### SET (SIMULATED) RUN DATE #####

sim_date = sim_date_list[sim_index] 
sim_date_ee = ee.Date(sim_date)
#For consistency, foramt the sim date the same as Discovery date as from pull_mtbs_fires.py
sim_date_formatted = str(sim_date)[0:10]
print(f'Simulated run date: {sim_date_formatted}')

# NOTE: Fires are filtered to the SAME date across all simulations
filter_date = sim_date_list[0]
filter_date_ee = ee.Date(filter_date)
filter_date_formatted = str(filter_date)[0:10]
print(f'Fire filter date: {filter_date_formatted}')

# capture the code run date (today's / current date) as well
code_date = datetime.datetime.now()
code_date_formatted = str(cur_dt)[0:10]

# Map over the fires, adding the simulated run date 
#  note: must use subfunction set_windows_sim() in bs calc function
#  Also add id field for labeling bands in exported intermediate products
def set_sim_date(feat: ee.Feature):
    fire_date = ee.Date(feat.getString('Discovery'))
    #find out if fire is after filter date (negative is before filter date)
    days_from_filter = fire_date.difference(filter_date_ee,'day')
    #Also want to know discovery in reference to simulation date
    #negative days mean Discovery was BEFORE simulated date
    days_from_sim = fire_date.difference(sim_date_ee,'day')
    #concatenate name and irwin ID
    fire_name = ee.String(feat.get('Incid_Name'))
    sep = ee.String('_')
    fire_irwin = ee.String(feat.get('irwinID'))
    fire_intermed = fire_name.cat(sep)
    fire_id = fire_intermed.cat(fire_irwin)
    return feat.set('sim_date',sim_date_formatted, 
                    'days_from_sim',days_from_sim,
                    'filter_date',filter_date_formatted,
                    'days_from_filter',days_from_filter,
                    'fire_id', fire_id,
                    'code_run_date',code_date_formatted)
fires_dt = fires.map(set_sim_date)

# Filter fires to only those that were discovered PRIOR to (negative days) the filter date
fires_sim = fires_dt.filter(ee.Filter.lt('days_from_filter', 0))

# Metadata collection and print
fires_sim_count = fires_sim.size().getInfo()
print(f'Fire count post simulated date filtering: {fires_sim_count}')


###### METADATA COLLECTION 2 ######
metadata = {
    'simulated_run_date':sim_date.strftime('%Y%m%d'),
    'code_run_date':code_date.strftime('%Y%m%d'),
    'fire_count_original':fires_count,
    'fire_count_final':fires_sim_count,
    'code_origin':'BS_Mapper_testing202308',
}
#print(metadata)


###### GET FIRE WINDOW DIAGNOSTICS ######

bs_windows = ee.FeatureCollection(fires_sim.map(af.bs_get_windows))
desc_windows = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_windows_{cur_dt.strftime("%Y%m%d")}'
#wanted properties
wanted_cols = ['irwinID', 'Incid_Name', 'filter_date', 'days_from_filter', 'Discovery', 
               'sim_date', 'days_from_sim', 
               'mode', 'pre_start', 'pre_end', 'post_start', 'post_end',
               'code_run_date']
task_csv = ee.batch.Export.table.toDrive(
    collection=bs_windows.set('code_run_date', cur_dt),
    description=desc_windows,
    selectors=wanted_cols,
    fileFormat='CSV')

if csv_export_flag == True:
    print(f'export task started: fire window diagnostics {desc_windows} to Google drive of user.')
    task_csv.start()


##### RUN BURN SEVERITY #####

#bs_test_feat = af.bs_calc_v2309(fires_sim.first())
#print(bs_test_feat.bandNames().getInfo())
#print(bs_test_feat.propertyNames().getInfo())

# Returns RdNBR & BS for each fire ee.Feature
bs_coll_combined = ee.FeatureCollection(fires_sim).map(af.bs_calc_v2309)
# print(bs_coll_combined.size().getInfo())

# Separate out RdNBR and Miller Threshold severity classes
bs_rdnbr = ee.ImageCollection(bs_coll_combined).select('RdNBR')
#print(bs_rdnbr.size().getInfo())
bs_coll = ee.ImageCollection(bs_coll_combined).select('MillersThresholds')
#print(bs_coll.size().getInfo())

    # print(bs_rdnbr.first().bandNames().getInfo()) #user memory error
    # print(bs_coll.first().bandNames().getInfo()) #user memory error


##### Export intermediate: individual features, RdNBR #####

# Turn each fire into bands of 1 image to export
# Get created property to name bands
rdnbr_band_ids = bs_rdnbr.aggregate_array('fire_id')
    #print(rdnbr_irwin_list.size().getInfo()) #user memory error
#to bands, rename, add metadata
rdnbr_banded = bs_rdnbr.toBands().rename(rdnbr_band_ids).set(metadata)

#set file name description with selected variables
desc_rdnbr = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_rdnbrbands_{cur_dt.strftime("%Y%m%d")}'
# Export Image Collection

utils.exportImgtoAsset(rdnbr_banded, 
                    desc=desc_rdnbr,
                    region=None, #note, not used for conus
                    asset_folder=asset_folder, 
                    export_type='conus',
                    export=export_flag)


# Export intermediate: individual features, severity classified
# Turn each fire into bands of 1 image to export
# Get created property to name bands
bs_band_ids = bs_coll.aggregate_array('fire_id')
#to bands, rename, add metadata
bs_banded = bs_coll.toBands().rename(bs_band_ids).set(metadata)
#set file name description with selected variables
desc_band = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_bsbands_{cur_dt.strftime("%Y%m%d")}'
# Export Image Collection

utils.exportImgtoAsset(bs_banded, 
                    desc=desc_band,
                    region=None, #note, not used for conus
                    asset_folder=asset_folder, 
                    export_type='conus',
                    export=export_flag)



##### COMPOSITE BURN SEVERITY & FINAL EXPORT #####

# composite that ee.ImageCollection with a max() reducer, add metadata
bs_composite = bs_coll.max().rename('SEVERITY').set('Year',int(yr)).set(metadata)

# To Asset
desc = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_{cur_dt.strftime("%Y%m%d")}'

utils.exportImgtoAsset(bs_composite, 
                    desc=desc,
                    region=None,
                    asset_folder=asset_folder, 
                    export_type='conus',
                    export=export_flag)



Total Fires in FeatureCollection: 4
Simulated run date: 2021-06-30
Fire filter date: 2020-09-30
Fire count post simulated date filtering: 3
export task started: fire window diagnostics bs_simrun2020_20210630_mtbs4_windows_20230911 to Google drive of user.
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/bs_simrun2020_20210630_mtbs4_rdnbrbands_20230911
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/bs_simrun2020_20210630_mtbs4_bsbands_20230911
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/bs_simrun2020_20210630_mtbs4_20230911


# For one fire 
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


In [ ]:
# Running for a particular fire

# Select the fire based on irwinID (or name and edit filter below)
#fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
#fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# Run burn severity tool 
bs_coll = ee.FeatureCollection(fires_f).map(af.bs_calc_new)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
bs_composite = ee.ImageCollection(bs_coll).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(bs_composite,  
                    desc=desc,
                    region=bs_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False) 


## RDNBR calculation only, as a test

In [ ]:
# Running for a particular fire - ONLY CALC RDNBR

# Select the fire based on irwinID 
    #fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
    #fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# CalC RDNBR
rdnbr_coll = ee.FeatureCollection(fires_f).map(af.rdnbr_only_calc)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
rdnbr_composite = ee.ImageCollection(rdnbr_coll).max().rename('RDNBR') 

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_rdnbr_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(rdnbr_composite,  
                    desc=desc,
                    region=rdnbr_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False)


# ee authentication block

In [ ]:
# import ee
# ee.Authenticate()